In [ ]:
#### The code is from Irene Yi-Ju Su ####
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/TEST")

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import torch, re
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, GPT2TokenizerFast, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
!pip3 install datasets
from datasets import load_dataset, Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
ds = load_dataset("sentence-transformers/eli5")
ds

README.md:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/113M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/325475 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 325475
    })
})

In [ ]:
temp = ds['train'].train_test_split(test_size=0.2, seed=42)
df_train, df_test = temp['train'].to_pandas(), temp['test'].to_pandas()

In [ ]:
def clean_text(text):
  if pd.isna(text): return ""
  if not isinstance(text, str): text = str(text)
  return re.sub(r'\s+', " ", text.lower().strip())
def df_prepare_dataset(df):
  df["question"], df["answer"] = df["question"].apply(clean_text), df["answer"].apply(clean_text)
  df = df[(df["question"] !="") & (df["answer"] !="")]
  return df
def clean_generated(generated):
  temp = re.sub(r'[\n|\\|\x0f-\x1f|\x7f-\xff]', "", generated)
  temp = re.sub(r'\s+', " ", temp).capitalize()
  temp = re.sub(r'\.{4,}', '...', temp)
  temp = re.sub(r'\.([a-zA-Z])', r'. \1', temp)
  temp = re.sub(r'(?<=[\.\?\!]\s)(\w)|^(\w)', lambda m: m.group().upper(), temp)
  return temp

In [ ]:
df_train, df_test = df_prepare_dataset(df_train), df_prepare_dataset(df_test)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
lengths1 = [len(tokenizer.encode(f"question: {q} answer: ")) for q in df_train["question"]]
lengths2 = [len(tokenizer.encode(f"question: {q} answer: {a}")) for q,a in zip(df_train["question"], df_train["answer"])]
print("Length stats for train data: question only")
print(pd.Series(lengths1).describe())
print("Length stats for train data: question + answer")
print(pd.Series(lengths2).describe())

Length stats for train data: question only
count    260380.000000
mean         23.720743
std          10.054201
min           7.000000
25%          17.000000
50%          21.000000
75%          28.000000
max         399.000000
dtype: float64
Length stats for train data: question + answer
count    260380.000000
mean        118.839369
std          56.358361
min          17.000000
25%          73.000000
50%         109.000000
75%         158.000000
max         760.000000
dtype: float64


In [ ]:
input_max_length, output_max_length = max(lengths1), int(np.percentile(lengths2, 90))
input_max_length, output_max_length

(399, 203)

In [ ]:
train_dataset, test_dataset = Dataset.from_pandas(df_train), Dataset.from_pandas(df_test)

In [ ]:
len(train_dataset)

260380

In [ ]:
def get_answer(question, model):
    device = next(model.parameters()).device
    input = tokenizer(
        f"question: {clean_text(question)} answer: ", #prompt
        return_tensors="pt",  #no need to pad. If pad make additional pairs, which result in worse result, waste memory, waste time
    ).to(device)
    output = model.generate(
        **input,
        do_sample = True, temperature = 0.7, top_p = 0.9,
        max_length = output_max_length, num_return_sequences = 1, no_repeat_ngram_size = 2, #no need to pad. Let model stop natually or at max_length
    )
    generated = tokenizer.decode(output[0], skip_special_tokens=True).split("answer:")[-1].strip() #only get tedt after "question: [question body] answer:"
    return clean_generated(generated)

In [ ]:
def prepare_dataset(x):
    return tokenizer(
        f'question: {x["question"]} answer: {x["answer"]}', #prompt
        return_tensors="np",
        padding = "max_length", max_length = output_max_length, truncation = True,
    )
train_dataset = train_dataset.map(prepare_dataset, remove_columns=['question', 'answer'])
test_dataset  = test_dataset.map(prepare_dataset,  remove_columns=['question', 'answer'])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/260380 [00:00<?, ? examples/s]

Map:   0%|          | 0/65095 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./MODEL/train_gpt2",
    logging_dir='./TEMP',
    overwrite_output_dir=True,

    dataloader_num_workers=4,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="steps",
    evaluation_strategy="steps",
    logging_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=500,

    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=2e-4,
    warmup_steps=500,
    lr_scheduler_type="cosine",

    fp16 = True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,3.797800,3.278596


Step,Training Loss,Validation Loss
500,3.797800,3.278596
1000,3.336000,3.217761
1500,3.283500,3.184777
2000,3.260600,3.164155
2500,3.233400,3.145819


In [ ]:
trainer.train(resume_from_checkpoint="./MODEL/train_gpt2/checkpoint-3500") #network disconnect, resume from last checkpoint

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
4000,3.190300,3.106978
4500,3.099000,3.104118
5000,3.076000,3.095425
5500,3.079700,3.087370
6000,3.069100,3.079429
6500,3.068800,3.071981
7000,3.061700,3.063911
7500,3.054100,3.059677
8000,3.048700,3.051771


Step,Training Loss,Validation Loss
4000,3.190300,3.106978
4500,3.099000,3.104118
5000,3.076000,3.095425
5500,3.079700,3.087370
6000,3.069100,3.079429
6500,3.068800,3.071981
7000,3.061700,3.063911
7500,3.054100,3.059677
8000,3.048700,3.051771
8500,2.988600,3.053135


In [ ]:
trainer.train(resume_from_checkpoint="./MODEL/train_gpt2/checkpoint-10000") #network disconnect, resume from last checkpoint

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
10500,2.968400,3.042934
11000,2.955400,3.041789


Step,Training Loss,Validation Loss
10500,2.968400,3.042934
11000,2.955400,3.041789
11500,2.957300,3.041485
12000,2.958700,3.041104


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=12204, training_loss=0.5344564812334199, metrics={'train_runtime': 1375.9612, 'train_samples_per_second': 567.705, 'train_steps_per_second': 8.869, 'total_flos': 8.0915173161984e+16, 'train_loss': 0.5344564812334199, 'epoch': 2.999877104583999})

In [ ]:
save_directory = "./MODEL/train_gpt2"
os.makedirs(save_directory, exist_ok=True)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
model.config.save_pretrained(save_directory)